In this notebook we are using dataset(20438 rows × 2 columns) after removing all null values as well as duplicate values. We are using pre trained distil bert transformers based model to train our dataset. We are tarining the model for 7 epochs, till 5 epochs we are getting(loss: 0.2972 - accuracy: 0.9018 - val_loss: 0.8151 - val_accuracy: 0.7987). For 6 & 7 epochs we are getting overfitting values(loss: 0.1751 - accuracy: 0.9401 - val_loss: 0.9539 - val_accuracy: 0.7943). 

In [ ]:
!nvidia-smi
#checking GPU configuration

Tue Jul 20 15:18:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 41.8 MB/s 
     |████████████████████████████████| 895 kB 33.5 MB/s 


In [ ]:
from google.colab import drive #mounting drve for file uploading
drive.mount('/content/data/')

Mounted at /content/data/


In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd

In [ ]:
df=pd.read_csv("/content/data/MyDrive/data/20krows.csv") # reading csv file  #consider only primary queries of students                                                    
df

,content,Intent
0,In this video as Sudhansu sir explains that we...,activation_function
1,I have some doubts regarding Activation functi...,activation_function
2,Suppose i am using any activation function and...,activation_function
3,I know in case vanishing gradient weight of in...,activation_function
4,I am a student of DLCVNLP 25th batch and have ...,activation_function
...,...,...
20433,I wanted to know how do you call while attendi...,zoom_platform_query
20434,DL Course today...zoom attendees reached 1000....,zoom_platform_query
20435,Zoom app was not working well aind even attend...,zoom_platform_query
20436,is their any change in settings that need to b...,zoom_platform_query


In [ ]:
df['type'] = df['Intent'].astype('category').cat.codes #create another column 'type' by encoding Intent column
df.head()

,content,Intent,type
0,In this video as Sudhansu sir explains that we...,activation_function,10
1,I have some doubts regarding Activation functi...,activation_function,10
2,Suppose i am using any activation function and...,activation_function,10
3,I know in case vanishing gradient weight of in...,activation_function,10
4,I am a student of DLCVNLP 25th batch and have ...,activation_function,10


In [ ]:
print(df["type"].min(),df["type"].max())

0 176


In [ ]:
X=list(df['content']) # Features (not-tokenized yet)
y=list(df['type'])# Lables

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
#DistillBert Tokenizer
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)#padding and truncation are using to make vectors of same size
val_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
#Loading data from Memory to Tensor Dataset and applying Transformation on it

import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),y_test))

In [ ]:
#DistillBert Training

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=177)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
#Training (Fine-Tuning)
model.fit(train_dataset.shuffle(1000).batch(8), epochs=7, batch_size=8, validation_data=val_dataset.shuffle(1000).batch(8))

Epoch 1/7
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
#Saving the model and the tokenizer
save_directory = "/content/data/MyDrive/data/models/20k_rows_disitil_bert/"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/data/MyDrive/data/models/20k_rows_disitil_bert/tokenizer_config.json',
 '/content/data/MyDrive/data/models/20k_rows_disitil_bert/special_tokens_map.json',
 '/content/data/MyDrive/data/models/20k_rows_disitil_bert/vocab.txt',
 '/content/data/MyDrive/data/models/20k_rows_disitil_bert/added_tokens.json')

In [ ]:
#Loading the model and the tokenizer
#from_pretrained() method to initialize a tokenizer or model, the library will download and cache files like the model weights file, tokenizer vocab file, and config files and save them locally.

loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /content/data/MyDrive/data/models/20k_rows_disitil_bert/ were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/data/MyDrive/data/models/20k_rows_disitil_bert/ and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infer

In [ ]:
test_text='what are the key points behind the distil bert'

In [ ]:
#Making Predictions with new data
predict_input = loaded_tokenizer.encode(test_text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

output = loaded_model(predict_input)[0]

prediction_value = tf.argmax(output, axis=1).numpy()[0]
df.loc[df['type'] == prediction_value, 'Intent'].iloc[0]

'Bert_query'